In [1]:
# Instalação das bibliotecas (execute apenas se não estiverem instaladas)
# !pip install gensim matplotlib scikit-learn pandas numpy spacy plotly

# Importações básicas
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from gensim.models import Word2Vec
import gensim.downloader as api
from sklearn.manifold import TSNE
import re
import nltk
from nltk.tokenize import word_tokenize
import warnings
warnings.filterwarnings('ignore')

# Download de recursos NLTK (se necessário)
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

ModuleNotFoundError: No module named 'gensim'

In [1]:
# Dados de exemplo - críticas de filmes (simplificadas)
textos = [
    "Este filme é incrível, adorei a atuação do protagonista",
    "A direção de fotografia é espetacular e o roteiro é envolvente",
    "Péssimo filme, desperdicei meu tempo assistindo isso",
    "Os atores são talentosos mas o roteiro é fraco",
    "Cinematografia belíssima, recomendo assistir no cinema",
    "Não gostei da história, personagens mal desenvolvidos",
    "A trilha sonora combina perfeitamente com as cenas",
    "Filme entediante, previsível do início ao fim",
    "Os efeitos especiais são impressionantes, tecnologia de ponta",
    "História emocionante, chorei no final do filme"
]

# Verificando os dados
for i, texto in enumerate(textos[:3]):  # Mostrando apenas os 3 primeiros
    print(f"Texto {i+1}: {texto}")

Texto 1: Este filme é incrível, adorei a atuação do protagonista
Texto 2: A direção de fotografia é espetacular e o roteiro é envolvente
Texto 3: Péssimo filme, desperdicei meu tempo assistindo isso


In [ ]:
from nltk.corpus import stopwords

def preprocessar_texto(texto):
    # Converter para minúsculas
    texto = texto.lower()

    # Remover caracteres especiais e números
    texto = re.sub(r'[^a-záàâãéèêíïóôõöúçñ ]', '', texto)

    # Tokenizar
    tokens = word_tokenize(texto)

    # Remover stopwords (opcional, dependendo da aplicação)
    stop_words = set(stopwords.words('portuguese'))
    tokens = [token for token in tokens if token not in stop_words]

    return tokens

# Aplicar pré-processamento a todos os textos
textos_preprocessados = [preprocessar_texto(texto) for texto in textos]

# Verificar resultado
print("Exemplo de texto original:")
print(textos[0])
print("\nDepois do pré-processamento:")
print(textos_preprocessados[0])

In [ ]:
# Definir parâmetros do modelo
vector_size = 100    # Dimensionalidade dos vetores
window = 5           # Tamanho da janela de contexto
min_count = 1        # Frequência mínima das palavras
workers = 4          # Número de threads para treinamento
sg = 1               # Modelo Skip-gram (1) ou CBOW (0)

# Treinar o modelo
model = Word2Vec(
    sentences=textos_preprocessados,
    vector_size=vector_size,
    window=window,
    min_count=min_count,
    workers=workers,
    sg=sg
)

print(f"Modelo treinado com {len(model.wv.key_to_index)} palavras no vocabulário")


In [ ]:
import spacy

# Carregar modelo spaCy (pequeno)
try:
    nlp = spacy.load("pt_core_news_sm")  # Para português
    # Alternativa para inglês: nlp = spacy.load("en_core_web_sm")

    # Exemplo de texto
    texto = "O banco está cheio de dinheiro. Eu sentei no banco da praça."

    # Processar o texto
    doc = nlp(texto)

    # Examinar embeddings para cada ocorrência da palavra "banco"
    for token in doc:
        if token.text.lower() == "banco":
            contexto = doc[max(0, token.i-3):min(len(doc), token.i+4)]
            print(f"Contexto: {contexto}")
            print(f"Vetor (10 primeiras dimensões): {token.vector[:10]}")
            print(f"Dimensão do vetor: {len(token.vector)}")
            print("-" * 50)
except Exception as e:
    print(f"Erro ao carregar spaCy: {e}")
    print("Talvez seja necessário instalar os modelos com:")
    print("python -m spacy download pt_core_news_sm")
    print("ou")
    print("python -m spacy download en_core_web_sm")


In [ ]:
from gensim.models import FastText

# Treinar modelo FastText
modelo_fasttext = FastText(
    sentences=textos_preprocessados,
    vector_size=100,
    window=5,
    min_count=1,
    workers=4,
    sg=1
)

# Testar com palavras que não estão no corpus
palavras_teste = ["filmagem", "cinematográfico", "atuações"]

print("\nEmbeddings para palavras fora do vocabulário (FastText):")
for palavra in palavras_teste:
    # Verificar se a palavra está no vocabulário original
    no_vocab = palavra in model.wv
    # Obter vetor do FastText (funciona mesmo para palavras fora do vocabulário)
    vetor = modelo_fasttext.wv[palavra]
    print(f"'{palavra}' (no vocabulário: {no_vocab})")
    print(f"Primeiras 5 dimensões do vetor: {vetor[:5]}")

    # Encontrar palavras similares
    similares = modelo_fasttext.wv.most_similar(palavra, topn=3)
    print(f"Palavras similares a '{palavra}':")
    for p, sim in similares:
        print(f"  {p}: {sim:.4f}")
    print()


In [ ]:
def similaridade_documentos(doc1, doc2, modelo):
    """Calcula a similaridade entre dois documentos usando embeddings"""
    vetor1 = texto_para_vetor(doc1, modelo)
    vetor2 = texto_para_vetor(doc2, modelo)

    # Calcular similaridade do cosseno
    # similaridade = 1 - distância do cosseno
    similaridade = np.dot(vetor1, vetor2) / (np.linalg.norm(vetor1) * np.linalg.norm(vetor2))
    return similaridade

# Exercício: Calcule a similaridade entre os documentos abaixo
documento1 = "O filme tem uma história envolvente e atuações convincentes"
documento2 = "A narrativa do filme é cativante e os atores são excelentes"
documento3 = "O restaurante tem comida deliciosa e preços acessíveis"

# Calcular similaridades (implemente sua solução)
# ...


In [ ]:
def detectar_topico(texto, palavras_chave_por_topico, modelo):
    """
    Detecta o tópico mais provável para um texto com base na similaridade de embeddings.

    Args:
        texto: Texto a ser classificado
        palavras_chave_por_topico: Dicionário {tópico: [palavras_chave]}
        modelo: Modelo de embeddings
    """
    # Vetorizar o texto
    vetor_texto = texto_para_vetor(texto, modelo)

    # Calcular similaridade média com cada conjunto de palavras-chave
    similaridades_topicos = {}

    # Implementar cálculo de similaridade média entre o texto e as palavras-chave
    # ...

    # Retornar o tópico com maior similaridade
    # ...

# Definir palavras-chave por tópico
topicos = {
    "Cinema": ["filme", "cinema", "ator", "diretor", "roteiro"],
    "Tecnologia": ["computador", "algoritmo", "software", "programação", "tecnologia"],
    "Esporte": ["futebol", "atleta", "equipe", "competição", "treino"]
}

# Textos para classificar
textos_para_classificar = [
    "O novo filme do diretor ganhou vários prêmios no festival",
    "A empresa lançou um software de inteligência artificial",
    "O time conquistou o campeonato após uma temporada difícil"
]

# Classificar textos (implemente sua solução)
# ...
